### Problem (Part II)

**Problem:** Cross selling means selling more products to a customer by analyzing their shopping trend and comparing the pattern of which to general shopping trends. In ecommerce, retailers will often offer customers bundle of products with attractive offers in order to boost sales.

**Dataset:** The online retail transactions dataset is available from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/online+retail). 

**Objective:** We want to understand what merchandise items customers purchase together and use that to offer additions to customers' original purchase as suggestion. 

**Approach:** Use **association rule-mining** technique for cross selling. More details will be covered below.

In [1]:
# Load Necessary Dependencies

import csv
import pandas as pd
import matplotlib.pyplot as plt
import Orange
from Orange.data import Domain, DiscreteVariable, ContinuousVariable
from orangecontrib.associate.fpgrowth import *

%matplotlib inline

### Part I: Exploratory Analysis

Load the Online Transaction Dataset

In [2]:
cs_mba = pd.read_excel(io=r'../data/Online Retail.xlsx')

# Focus only on transactions in UK & non-refund transactions
cs_mba_uk = cs_mba[cs_mba.Country == 'United Kingdom']
cs_mba_uk = cs_mba_uk[~(cs_mba_uk.InvoiceNo.str.contains("C") == True)]
cs_mba_uk = cs_mba_uk[~cs_mba_uk.Quantity<0]

In [3]:
cs_mba_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
print(f'The dataset contains {cs_mba_uk.shape[0]} transactions.')
print(f'It contains transactions on {cs_mba_uk.StockCode.unique().shape[0]} items.') 

The dataset contains 486286 transactions.
It contains transactions on 3936 items.


### Part II Data Preparation

**FP growth algorithm:** The algorithm we use for this association-rule mining task. It uses a special data structure called FP Tree to hold itemset association information and uses a divide-and-conquer strategy to find frequent itemsets without generating all itemsets (${2^k}$ sets if there are k items).

![title](fp_growth.png)

**Orange:** The framework we use for implementing FP growth and core data structures needed. Basically we need convert panads dataframe to *Orange table* data structure.

**Input data format**: two dimensional matrix, where each row represents a transaction and each column represents an item. The cell value - 0 or 1 - means whether the item appears in the transaction.

The first step is to convert our transaction dataframe to the required format.

In [14]:
def reformat_df(raw_df):
    # Gather a list of items
    items = list(raw_df.Description.unique())
    
    # Group transactions by invoice number because
    # the same invoice means these items are purchased together
    transaction_level_df = \
        raw_df.groupby('InvoiceNo'). \
        aggregate(lambda x: tuple(x)). \
        reset_index()[['InvoiceNo','Description']]
    
    # Create a new dataframe to store these results
    output_dict = dict()
    rows = []
    # Loop through each row to perform the transformation
    for rec in transaction_level_df.to_dict('records'):
        invoice_num, items_list = rec['InvoiceNo'], rec['Description']
        transaction_dict = {item:0 for item in items}
        # Mark 1 when the item exists
        transaction_dict.update({item:1 for item in items if item in items_list})
        rows.append(transaction_dict)
    
    transaction_df = pd.DataFrame(rows) 
    return transaction_df

In [15]:
tranasction_df = reformat_df(cs_mba_uk)
tranasction_df.head(3)

,WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL LANTERN,CREAM CUPID HEARTS COAT HANGER,KNITTED UNION FLAG HOT WATER BOTTLE,RED WOOLLY HOTTIE WHITE HEART.,SET 7 BABUSHKA NESTING BOXES,GLASS STAR FROSTED T-LIGHT HOLDER,HAND WARMER UNION JACK,HAND WARMER RED POLKA DOT,ASSORTED COLOUR BIRD ORNAMENT,...,SET OF 6 RIBBONS COUNTRY STYLE,SNACK TRAY RED VINTAGE DOILY,SET OF 6 RIBBONS PERFECTLY PRETTY,SET OF 6 RIBBONS PARTY,SET 10 CARDS SNOWY ROBIN 17099,SET 10 CARDS SWIRLY XMAS TREE 17104,"LETTER ""U"" BLING KEY RING",CREAM HANGING HEART T-LIGHT HOLDER,BLACK SIL'T SQU CANDLE PLATE,"PAPER CRAFT , LITTLE BIRDIE"
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


#### Data pruning

In order to make the algorithm perform well, we need to ensure the merchandise items have appeared frequently enough in our transactions. Thi is because frequent items are more likely to appear in co-purchase patterns. On the other hand, data pruning also reduce the size of transactions, which shortens the runtime of the algorirhm. In this case, we set **threshold = 0.4** which means we only retain items that explain 50% of sales, from the most frequent items to the least.

In [17]:
THRESHOLD = 0.4
def prune_dataset(input_df, 
                  length_trans=2, 
                  total_sales_perc=0.5, 
                  start_item=None, 
                  end_item=None):
    if 'total_items' in input_df.columns:
        del(input_df['total_items'])
    # Count items 
    item_count = input_df.sum().sort_values(ascending = False).reset_index()
    total_items = sum(input_df.sum().sort_values(ascending = False))
    item_count.rename(columns={item_count.columns[0]:'item_name',item_count.columns[1]:'item_count'}, inplace=True)
    if not start_item and not end_item: 
        # The item frequency is sorted. Compare cumulative sum of 
        # percentage with the threshold, and identify the items whose 
        # cumulative sum is barely smaller than the threshold.
        item_count['item_perc'] = item_count['item_count']/total_items
        item_count['total_perc'] = item_count.item_perc.cumsum()
        selected_items = list(item_count[item_count.total_perc < total_sales_perc].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items], item_count[item_count.total_perc < total_sales_perc]
    elif end_item > start_item:
        selected_items = list(item_count[start_item:end_item].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items],item_count[start_item:end_item]

In [18]:
output_df_uk_n, item_counts_n = prune_dataset(input_df=tranasction_df, 
                                              length_trans=2,
                                              total_sales_perc=THRESHOLD,
                                              start_item=0, 
                                              end_item=15)
print(f'After pruning the dataset is left {output_df_uk_n.shape} rows.')

After pruning the dataset is left (4961, 15) rows.


Association Rule Mining with FP Growth